<a href="https://colab.research.google.com/github/hrbolek/learning/blob/master/operanalyst/dynprog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dynamické programování

## Princip optimality

Dynamické programování je motoda operační analýzy, která je určena pro optimalizaci problému, který lze rozdělit do etap. Předpokládejme, že těchto etap je $n$. Potom při řízení můžeme v každé etapě použít řízení $u_i$, čímž ovlivňujeme vývoj stavu řízeného systému. Tento systém nabývá postupně stavů $x_i$. Z matematického hlediska jak řízení $u_i$, tak stavy $x_i$ mohou být vektory.

Cílem je optimalizovat účelovou funkci 

$$J=\sum_{i=0}^{n-1}I_i$$

kde $J$ a $I_i$ jsou např. spotřeba, náklady, apod.

Optimálním řešením je posloupnost vektorů $u=(u_0^*, u_1^*, \dots, u_n^*)$.

R. Bellman formuloval princip optimality: 
> "Optimální strategie hledá optimální pokračování operace vzhledem ke stavu, který nastal v důsledku přijetí předchozího rozhodnutí."

